In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import zipfile
import os

#zip_path = '/content/drive/MyDrive/data.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [ ]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models

CLASS_NAMES = [
    "orange_2x2_n", "blue_4x1_n", "white_4x2_n", "orange_1x1_s", "red_4x1_n",
    "blue_4x2_n", "orange_4x1_n", "white_2x2_n", "red_4x2_n", "blue_2x2_n",
    "white_4x1_n", "red_1x1_s", "orange_4x2_n", "white_1x1_s", "red_2x2_n"
]
MAPS = {name: i for i, name in enumerate(CLASS_NAMES)}

In [ ]:
class LegoDataset(Dataset):
    def __init__(self, json_path, img_dir, transform=None):
        with open(json_path, 'r') as f:
            raw_data = json.load(f)

        self.data = []
        for item in raw_data:
            label = item['annotations'][0]['result'][0]['value']['choices'][0]
            self.data.append({"img": item['data']['image'].split('/')[-1], "label": label})

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_path = os.path.join(self.img_dir, item['img'])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(MAPS[item['label']])

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
class LegoClassificationModel(nn.Module):
    def __init__(self, num_classes=15):
        super().__init__()
        self.backbone = models.resnet18(weights='DEFAULT')
        num_fts = self.backbone.fc.in_features

        self.backbone.fc = nn.Linear(num_fts, num_classes)

    def forward(self, x):
        return self.backbone(x)

In [ ]:
class LegoDataset(Dataset):
    def __init__(self, json_path, img_dir, transform=None):
        with open(json_path, 'r') as f:
            raw_data = json.load(f)

        self.data = []
        for item in raw_data:
            label = item.get('brick_type')

            if 'image' in item:
                img_name = item['image'].split('/')[-1]
                actual_img_name = img_name.split('-', 1)[-1] if '-' in img_name else img_name

                if label:
                    self.data.append({"img": actual_img_name, "label": label})

        print(f"JSON-MIN Loaded: {len(self.data)} labeled images found.")
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_path = os.path.join(self.img_dir, item['img'])


        image = Image.open(img_path).convert("RGB")

        label_idx = torch.tensor(MAPS[item['label']])

        if self.transform:
            image = self.transform(image)
        return image, label_idx

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LegoClassificationModel(num_classes=15).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()


full_dataset = LegoDataset('/content/data/labels.json', '/content/data', transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f" Data ready: {train_size} training images, {val_size} validation images.")

In [ ]:
epochs = 25

for epoch in range(epochs):

    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)


    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Val Acc: {val_acc:.2f}%")

print("Training Complete!")

In [ ]:

save_path = '/content/drive/MyDrive/lego_sorter.pth'
torch.save(model.state_dict(), save_path)
print(f" Model saved to: {save_path}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LegoClassificationModel(num_classes=15).to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/lego_sorter.pth', map_location=device))
model.eval()
print(" Blueprint and Weights Restored!")

In [ ]:
import torch
from PIL import Image

CLASS_NAMES = [
    "orange_2x2_n", "blue_4x1_n", "white_4x2_n", "orange_1x1_s", "red_4x1_n",
    "blue_4x2_n", "orange_4x1_n", "white_2x2_n", "red_4x2_n", "blue_2x2_n",
    "white_4x1_n", "red_1x1_s", "orange_4x2_n", "white_1x1_s", "red_2x2_n"
]

def test_prediction(img_path):
    model.eval()

    image = Image.open(img_path).convert("RGB")


    input_tensor = transform(image).unsqueeze(0).to(device)


    with torch.no_grad():
        output = model(input_tensor)


    probabilities = torch.nn.functional.softmax(output, dim=1)
    confidence, predicted_idx = torch.max(probabilities, 1)


    prediction = CLASS_NAMES[predicted_idx.item()]

    return {
        "Prediction": prediction,
        "Confidence": f"{confidence.item() * 100:.2f}%"
    }

# Run test on your image
image_path = '/content/brick_12345678.jpg'
print(test_prediction(image_path))